Components of the Entire Python3 program:

1. Find the numerical solutions, cn, for any "n" constants (represented by zn) and delta "d."

2. Define the polynomial as a list of coefficients.

3. Find close solutions. When we obtain solutions of the polynomial equations, we are only interested in the previous element of the sequence
   This part of the code returns the "closest" solutions.

4. Setting up a recursion.

5. Checking the solution is close to zero.

6. Checking that all solutions are different.

7. Define A(z1, z2) function. 

8. Define B1 and B2 function. 

9. Define a function for terms inside the summation. 

10. Define a function for the summation. 

11. We check the identity. 

12. Plotting the soltuion. 

In [143]:
#Import libraries for plotting (matplotlib) and advanced mathematical operations (using numpy)
import numpy as np
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')

C:\Users\coe_user\AppData\Local\Temp\ipykernel_10788\1170155103.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-poster')


In [144]:
#Part 1: Find the numerical solutions for any n constants

#FUNCTION DEFINITION: Accepts parameters delta "d" and *z as pointer allowing for any number of z values 
def compute_coefficient(d, *z):
    n = len(z)

    is_even = n % 2 == 0 #if there is no remainder when divided by two, is_even is set to true. Otherwise, set to false.

    #create an empty list to store the coefficients 
    coeffs = []

    for i in range(n):
        #initalize numerator and denominator for each element in the computation before updating the values in the second loop 
        numerator = 1
        denominator = 1
        
        for j in range(n):

            #compute the numerical solutions for the Bethe equations for the numerator and denominator 
            numerator *= (1 + z[i]*z[j] - d*z[i])
            denominator *= (1 + z[j]*z[i] - d*z[j])

        #append numerical solutions to the result and return as apart of the list
        if is_even:
            coeffs.append(-numerator/denominator) #when n is even, sign should be negative
        else:
             coeffs.append(numerator/denominator) #when n is odd, sign should be positive

    return coeffs

#FUNCTION CALL : Initialize list z and int d, store returned list and print 
print("\nTest examples for n = 2 and n = 4 cases : \n")

#Works for when 'z' contains two elements
z2 = [2, 3]
d2 = 4

coeffs2 = compute_coefficient(d2, *z2)
print(coeffs2)

#Check: Product of coefficients (all the elements in the list) should sum to one 
from functools import reduce
import operator
product2 = reduce(operator.mul, coeffs2)
print(product2)

print("\n")

#Works for when 'z' contains four elements
z4 = [2, 3, 4, 5]
d4 = 6

coeffs4 = compute_coefficient(d4, *z4)
print(coeffs4)

product4 = reduce(operator.mul, coeffs4)
print(product4)


Test examples for n = 2 and n = 4 cases : 

[-0.2, -5.0]
1.0


[-0.004784688995215311, -0.14285714285714285, -3.6666666666666665, -399.0]
1.0


In [145]:

"""
Notes: For each coefficient, you want to create the list [1, 0, ..., 0, ci]
       Corresponds to z1L - c1 = 0, . . .,
       L roots depend on the ith coefficient 
       The solution of the polynomial will be stored in a Result list of length L with the roots 
       Once we have the solution above, we want to pick the closest solution 
"""

#Part 2: Define the polynomial as the list of coefficients 
def pol_coeffs(l, d, *z):

    c = compute_coefficient(d, *z) #c is a list that stores all the coefficients 
    #convert the list of n numbers to a list of n lists ; each number becomes its own list; each list is length l + 1
    #first element of the list will be 1, last will be ci, everything in between is zero 

    coeffs = [] #list of lists 
    results = [1]

    for i in c:
        
        for j in range (l-1):

            results.append(0)

        results.append(-i)
        coeffs.append(results) #attempt at appending for a specific coefficient (error message here, why?)
        results = [1] #creates a list of length l + 1  
        
    return coeffs

#test example from BAEsolutionsN2.ipynb 
print("\nTest example from BAE solutions.ipynb : \n")
l1 = 5
d1 = 4
z1 = [2, 3]
print(pol_coeffs(l1, d1, *z1))

print("\n")
l2 = 2
d2 = 0
z2 = [1, 1]
print(pol_coeffs(l2, d2, *z2))


Test example from BAE solutions.ipynb : 

[[1, 0, 0, 0, 0, 0.2], [1, 0, 0, 0, 0, 5.0]]


[[1, 0, 1.0], [1, 0, 1.0]]


In [146]:
#Part 3: Finding close solutions

"""
List of lists (for each sublists) -> array of roots -> pick the closest one 
Go back from n lists to n numbers 
"""

def close_sols(l, *zs, d, s):
    
    result = [] #empty list to store the closest solution for each coefficient  

    #iterate over each sublist 
    for sublist in pol_coeffs(l, *zs, d):

        #calculate the roots
        roots = (np.roots(sublist))

        #move to the previous element zi  
        roots = [x-s for x in roots]  

        #find the solution with the minimum absolute value and add s
        min_sol = min(roots, key=np.abs) + s  

        #results should be n numbers
        result.append(min_sol)

    return result  #Possible error of output not matching N = 2 case

cs_z = [1j, -1j]
cs_d = 0.1
cs_s = 1j
cs_l = 2 

print("\nPrinting closed solutions when 'cs_l = 2, cs_z = [1j, -1j], cs_d = 0.1, and cs_s = 1j': \n")
print(close_sols(cs_l, *cs_z, d=cs_d, s=cs_s))


Printing closed solutions when 'cs_l = 2, cs_z = [1j, -1j], cs_d = 0.1, and cs_s = 1j': 

[(0.19852871119604185+0.24216609671861722j), (-2.024603544794088+2.4696193053976527j)]


In [147]:
#Part 4: Setting up a recursion 

trials = 30

def sol_sys(l, d, *ks, trials):
    z = []
    for sublist in pol_coeffs(l, 0, 0, 0):
        sol = np.roots(sublist)
        z.append(sol)

    for _ in range(trials):
        z = [close_sols(l, *ks, d=d, s=s)]

    return z

l = 5
s = 2
ks = [1, 2]
print("\nPrinting the solutions to the system:", sol_sys(l, 0.1, *ks, trials=trials))



Printing the solutions to the system: [[(0.9382951259780539+0j), (1.0657627566354742+0j)]]


In [148]:
#Part 5: Checking The Solution (Showing that all Solutions 'z' are Close to Zero) 
l = 5
d = 0.1
ks = [1, 4]
trials = 30

z = sol_sys(l, d, *ks, trials=trials)
c = compute_coefficient(*ks, d)

print("\nChecking solutions are close to zero:")

#print z[i] ** l minus coefficient[i] 
for sublist, coefficient in zip(z, c):
    for element in sublist:
        print(element**l - coefficient, ",")



Checking solutions are close to zero:
(-3.3306690738754696e-15+0j) ,
(-1.5000000000000004+0j) ,


In [149]:
#Part 6: Checking the Solution (Showing that all Solutions 'z' are Different) 
l = 5
d = .1
trails = 300
all_sol = [sol_sys(l, d, *ks, trials = trials) for k1 in range(l) for k2 in range(l) if k1 != k2]
print("\nPrinting all solutions: ", all_sol)

diff = [abs(x[0]-y[0])+abs(x[1]-y[1]) for x in all_sol for y in all_sol if x != y]
diff.sort()
print("\nPrinting different solutions: ", diff)


Printing all solutions:  [[[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [[(1.1486983549970347+0j), (0.8705505632961241+0j)]], [

In [150]:
#Part 7: Checking the Initial Conditions 

def A(z1, z2, d):
    return (-1)* (1 + z1*z2 - d*z2)/(1 + z1 *z2 - d*z1)

print("\nChecking initial condition A: ", A(1,2,4)) #expected -5.0



Checking initial condition A:  -5.0


In [151]:
#Part 8: Define B1 B2 Function


In [152]:
#Part 9: Define a Function for terms inside the Summation

In [153]:
#Part 10: Define a Function for the Summation

In [154]:
#Part 11: We Check the Identity 

In [155]:
#Part 12: Plotting the Solution